# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import geoviews as gv
import geoviews.feature as gf
from geoviews import dim, opts
from cartopy import crs
import geoviews.tile_sources as gts


# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hami,42.8000,93.4500,28.24,13,26,4.16,CN,1717216071
1,1,adamstown,-25.0660,-130.1015,20.25,63,85,2.40,PN,1717216071
2,2,college,64.8569,-147.8028,20.99,36,75,5.66,US,1717216071
3,3,port-aux-francais,-49.3500,70.2167,4.67,77,96,14.03,TF,1717216071
4,4,koungou,-12.7336,45.2042,26.84,78,20,7.72,YT,1717216071


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
geoview_dataset = gv.Dataset(city_data_df, kdims=['City', 'Humidity'])
points = geoview_dataset.to(gv.Points, ['Lng', 'Lat'], ['City', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed'])
osm_map = gv.tile_sources.OSM
osm_map.opts(width=1200, height=700) * points.opts(tools=['hover'], size=dim('Humidity')*0.15, color='black', fill_color='City', cmap='gist_rainbow', alpha=0.6)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Humidity,Cloudiness,Wind Speed)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[city_data_df['Max Temp'].le(27) & city_data_df['Max Temp'].ge(21) &
                                city_data_df['Wind Speed'].le(4.5) &
                                city_data_df['Cloudiness'].eq(0) ].copy()

# Drop any rows with null values
filtered_cities.dropna(inplace=True)

# Display sample data
print(filtered_cities.shape)
filtered_cities.head()

(6, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,al kharijah,25.4514,30.5464,25.44,25,0,3.83,EG,1717216076
105,105,griffith,41.5284,-87.4237,21.96,66,0,1.54,US,1717216089
110,110,ibshaway,29.3580,30.6814,25.76,29,0,1.63,EG,1717216090
263,263,al bawiti,28.3492,28.8659,25.23,28,0,2.97,EG,1717216115
439,439,natal,-5.7950,-35.2094,25.12,88,0,3.09,BR,1717216146


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
34,al kharijah,EG,25.4514,30.5464,25,
105,griffith,US,41.5284,-87.4237,66,
110,ibshaway,EG,29.3580,30.6814,29,
263,al bawiti,EG,28.3492,28.8659,28,
439,natal,BR,-5.7950,-35.2094,88,


In [12]:
params["bias"]

Object '["bias"]' not found in the namespace.


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'filter' : {'lat' : '',
                'lon' : '',
                'search_rad_meters' : 10000,
                'category' : 'accommodation.hotel',
                'constructed_filter' : ''
                },
    'bias' : ''
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row[2]
    lon = row[3]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"]['lat'] = lat
    params["filter"]['lon'] = lon
    params['filter']['constructed_filter'] = f'categories={params['filter']['category']}&filter=circle:{params["filter"]['lon']},{params["filter"]['lat']},{params["filter"]['search_rad_meters']}&apiKey={geoapify_key}'

    params["bias"] = f'bias=proximity:{params["filter"]['lon']},{params["filter"]['lat']}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(f'{base_url}?{params['filter']['constructed_filter']}&{params["bias"]}')
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")



# Display sample data
hotel_df

Starting hotel search


/tmp/ipykernel_16741/1740037471.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lat = row[2]
/tmp/ipykernel_16741/1740037471.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lon = row[3]


al kharijah - nearest hotel: Kharga Hotel


/tmp/ipykernel_16741/1740037471.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lat = row[2]
/tmp/ipykernel_16741/1740037471.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lon = row[3]


griffith - nearest hotel: Quality Inn & Suites Hammond


...

/tmp/ipykernel_16741/1740037471.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lat = row[2]
/tmp/ipykernel_16741/1740037471.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lon = row[3]


natal - nearest hotel: Natal Palace Hotel


/tmp/ipykernel_16741/1740037471.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lat = row[2]
/tmp/ipykernel_16741/1740037471.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lon = row[3]


umluj - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
34,al kharijah,EG,25.4514,30.5464,25,Kharga Hotel
105,griffith,US,41.5284,-87.4237,66,Quality Inn & Suites Hammond
110,ibshaway,EG,29.3580,30.6814,29,No hotel found
263,al bawiti,EG,28.3492,28.8659,28,Old Oasis Hotel
439,natal,BR,-5.7950,-35.2094,88,Natal Palace Hotel
447,umluj,SA,25.0213,37.2685,61,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
geoview_dataset = gv.Dataset(hotel_df, kdims=['City', 'Humidity'])
points = geoview_dataset.to(gv.Points, ['Lng', 'Lat'], ['Country', 'City', 'Hotel Name', 'Humidity'])
osm_map = gv.tile_sources.OSM
#Changed humidity scale to make the smaller values in this plot more readable.
osm_map.opts(width=1200, height=700) * points.opts(tools=['hover'], size=dim('Humidity')*0.2, color='black', fill_color='City', cmap='gist_rainbow', alpha=0.6)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,City,Hotel Name,Humidity)